In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
# from tensorflow.python.compiler.mlcompute import mlcompute
# mlcompute.set_mlc_device(device_name='any')


In [4]:
path_to_file = 'shakespeare.txt'

In [5]:
text = open(path_to_file, 'r').read()

In [6]:
text[:500]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [7]:
vocab = sorted(set(text))

In [8]:
len(vocab)

84

In [9]:
# for pair in enumerate(vocab):
#     print(pair)

In [10]:
char_to_ind = {char:ind for ind, char in enumerate(vocab)}

In [11]:
char_to_ind['H']

33

In [12]:
index_to_char = np.array(vocab)

In [13]:
index_to_char[33]

'H'

In [14]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [15]:
encoded_text.shape

(5445609,)

In [16]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [17]:
line = "From fairest creatures we desire increase,"

In [18]:
len(line)

42

In [19]:
lines = '''
From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
'''

In [20]:
len(lines)

133

In [21]:
seq_len = 120

In [22]:
tota_num_seq = len(text) // (seq_len+1)

In [23]:
tota_num_seq

45005

In [24]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [25]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [26]:
# for item in char_dataset.take(500):
#     print(index_to_char[item.numpy()])

In [27]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [28]:
def create_seq_targets(seq):
    input_text = seq[:-1]
    target_text = seq[1:]
    return input_text, target_text

In [29]:
datasets = sequences.map(create_seq_targets)

In [30]:
for input_text, target_text in datasets.take(1):
    print(input_text.numpy())
    print("".join(index_to_char[input_text.numpy()]))
    print('\n')
    print(target_text.numpy())
    print("".join(index_to_char[target_text.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [31]:
batch_size = 128

In [32]:
buffer_size = 10000
datasets = datasets.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [33]:
datasets

<BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int32, name=None), TensorSpec(shape=(128, 120), dtype=tf.int32, name=None))>

In [34]:
vocab_size = len(vocab)

In [35]:
embed_dim = 64

In [36]:
rnn_neurons = 1026

In [37]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [38]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding

In [40]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons, return_sequences=True,stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss)
    return model

In [41]:
model = create_model(vocab_size=vocab_size, embed_dim=embed_dim, rnn_neurons=rnn_neurons, batch_size=batch_size)

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 64)           5376      
                                                                 
 gru (GRU)                   (128, None, 1026)         3361176   
                                                                 
 dense (Dense)               (128, None, 84)           86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [43]:
for input_example_batch, target_example_batch in datasets.take(1):
    example_batch_predictions = model(input_example_batch)

In [44]:
example_batch_predictions.shape

TensorShape([128, 120, 84])

In [45]:
sample_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [46]:
sample_indices = tf.squeeze(sample_indices, axis=-1).numpy()

In [47]:
sample_indices

array([11, 53, 50, 33, 61, 56, 25, 14, 27,  5, 41,  5, 46, 39, 69, 22, 52,
       52, 74, 78, 51, 22, 33, 22, 43, 16, 13, 45, 17, 21, 31, 79, 29, 40,
       18, 74, 30, 29, 34, 69, 72, 78, 45, 46,  6, 35, 69, 74, 70, 23, 61,
       43, 50, 50, 82, 53, 48, 45, 45, 52, 77, 12, 13, 68, 69, 31, 39, 46,
       40, 46, 45,  5, 48,  6, 64, 14, 51,  5, 62, 33, 31, 38, 66, 16, 32,
       55, 13, 65, 79, 51, 28, 69, 77,  1, 78, 33, 71, 40, 25, 31, 72, 56,
       80, 17, 50, 26,  7, 70, 30, 33,  7, 45, 12, 27, 21, 66, 31, 77,  2,
        0], dtype=int64)

In [48]:
index_to_char[sample_indices]

array(['0', ']', 'Y', 'H', 'f', 'a', '?', '3', 'B', "'", 'P', "'", 'U',
       'N', 'n', ';', '[', '[', 's', 'w', 'Z', ';', 'H', ';', 'R', '5',
       '2', 'T', '6', ':', 'F', 'x', 'D', 'O', '7', 's', 'E', 'D', 'I',
       'n', 'q', 'w', 'T', 'U', '(', 'J', 'n', 's', 'o', '<', 'f', 'R',
       'Y', 'Y', '|', ']', 'W', 'T', 'T', '[', 'v', '1', '2', 'm', 'n',
       'F', 'N', 'U', 'O', 'U', 'T', "'", 'W', '(', 'i', '3', 'Z', "'",
       'g', 'H', 'F', 'M', 'k', '5', 'G', '`', '2', 'j', 'x', 'Z', 'C',
       'n', 'v', ' ', 'w', 'H', 'p', 'O', '?', 'F', 'q', 'a', 'y', '6',
       'Y', 'A', ')', 'o', 'E', 'H', ')', 'T', '1', 'B', ':', 'k', 'F',
       'v', '!', '\n'], dtype='<U1')

In [49]:
epochs = 30

In [50]:
# temp = list(datasets.as_numpy_iterator())

In [ ]:
model.fit(datasets, epochs=epochs)

In [53]:
# model.save("nlp.h5")

In [54]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights('nlp.h5')
model.build(tf.TensorShape([1,None]))

In [55]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_1 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_1 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [57]:
def generate_text(model, start_seed,gen_size=500, temp=1.0):
  num_generate = gen_size
  input_eval = [char_to_ind[s] for s in start_seed]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = temp
  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(index_to_char[predicted_id])
  return start_seed + "".join(text_generated)


In [58]:
print(generate_text(model, "JULIET", gen_size=1000))

JULIETCK. It's butied for, I kill'd.            Exit
  PROSPERO. Ay, in the heavy of you,
    Sir England, what have we.
    The rather for his golden are as second manner so inlawartly, them to pluck one enoughs; I speak
    me. Yonder the thoughts
    Like to a milk and that adulter deep,
    And say thou art an old luce, weav'h
    In the remembrance of my daughter's death.
    I speak my mistress.
  ANTONIO. Why, villain, art thou domandon?
  JULIA. A dispossible advice it stays ithere    Turn this Captain Bourn's mind.      Exeunt.




Scene III.
A lintth of Archbishops

    F Simplicidy were but hearing of his knees
    Give thee him.'
    He has backward and uncurrent days her]
     That Lantter to have the third time but their
    fortune.
  PAINTER. Fe brat of me?
  Bene. Why let it be?
  MIRANDA. I will not lie when I have so beholding to
    your city hast thou touch'd the realm.
  Lear. Not I turn th' Least of our friends. Go to, Peter- I confess it well.
  COSTARD. I would